# Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importing Helper Functions
from helper_functions import drop_extraneous_col
# Recursive Feature Elimination with Cross-Validation
from sklearn.feature_selection import RFECV
# Time Series Split and GridSearchCV, where GridSearchCV is for hyperparameter tuning
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_validate, RandomizedSearchCV
# Pipeline
from sklearn.pipeline import Pipeline
# Standard Scalar
from sklearn.preprocessing import StandardScaler
# Confusion Matrix
from sklearn.metrics import confusion_matrix, make_scorer, recall_score, precision_score, accuracy_score, f1_score, roc_auc_score
# Logistic Regression, Ridge Classifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# XGBoost
from xgboost import XGBClassifier
# Support Vector Machine (SVM)
from sklearn.svm import SVC

# Using the Sliding Window Average DataFrame

In [ ]:
training_df  = pd.read_csv('csvs/.csv')
drop_extraneous_col(training_df)
training_df

# Splitting Dataframe into Train and Test

In [ ]:
undesired_columns = ['team0', 'team1', 'winner', 'season', 'date', 'team1_winner']
# We decided to train from the 2018 season to the 2023 season
training_seasons = [2018,2019,2020,2021,2022,2023]
# Splitting the dataframe into train and test
X_train = training_df[training_df['season'].isin(training_seasons)].drop(undesired_columns, axis=1)
X_test = training_df[training_df['season'] == 2024].drop(undesired_columns, axis=1)
y_train = training_df[training_df['season'].isin(training_seasons)]['team1_winner']
y_test = training_df[training_df['season'] == 2024]['team1_winner']


In [ ]:
# Double checking the shapes of the training and testing dataframes

print(f'Observations in X_train: {X_train.shape[0]}')
print(f'Observations in y_train: {y_train.shape[0]}')

print(f'Observations in X_test: {X_test.shape[0]}')
print(f'Observations in y_test: {y_test.shape[0]}')


# Scaling Features

In [ ]:
std_scalar = StandardScaler()
X_train = std_scalar.fit_transform(X_train)
X_test = std_scalar.fit_transform(X_test) 

# Defining the Type of Cross Validation

In [ ]:
# Define the type of cross validation
tscv = TimeSeriesSplit()

# Creating a Results DataFrame to Store Training and Validation Scores

In [ ]:
results_df = pd.DataFrame(columns=['Model', 'Training Accuracy', 'Validation Accuracy',
                             'Training Precision', 'Validation Precision',
                             'Training Recall ', 'Validation Recall',
                             'Training F1', 'Validation F1',
                             'Training ROC_AUC', 'Validation ROC_AUC'])

# Logistic Regression

In [ ]:
# Define the Logistic Regresion Model
model_lr = LogisticRegression(solver='saga', max_iter=5000, random_state=42)
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_lr = RFECV(estimator = model_lr, cv=tscv, min_features_to_select=30, scoring='accuracy')
# Fitting the rfecv model to the data
model_rfecv_lr.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_lr.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

grid_search_lr = GridSearchCV(estimator=model_lr, param_grid=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
grid_search_lr.fit(X_train_selected,y_train)

cv_results_lr = grid_search_lr.cv_results_

new_row_data = ['Logistic Regression']
for scorer in scoring:
    best_validation_score = cv_results_lr[f'mean_test_{scorer}'].max()
    i = list(cv_results_lr[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_lr[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

# Random Forest Classifier

In [ ]:
# Define the RandomForestClassifier Model
model_rfc = RandomForestClassifier(random_state=42)
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_rfc = RFECV(estimator = model_rfc, cv=tscv, min_features_to_select=30, scoring='accuracy')
# Fitting the rfecv model to the data
model_rfecv_rfc.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_rfc.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'n_estimators': [100, 200, 500],       # Number of trees in the forest.
    'max_depth': [None, 10, 20, 30],        # Maximum depth of the tree.
    'min_samples_split': [2, 5, 10],        # Minimum number of samples required to split an internal node.
    'min_samples_leaf': [1, 2, 4], 
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

random_search_rfc = RandomizedSearchCV(estimator=model_rfc, n_iter=100, param_distributions=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
random_search_rfc.fit(X_train_selected,y_train)

cv_results_rfc = random_search_rfc.cv_results_

new_row_data = ['Random Forest Classifier']
for scorer in scoring:
    best_validation_score = cv_results_rfc[f'mean_test_{scorer}'].max()
    i = list(cv_results_rfc[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_rfc[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

# XGBoost Classifier

In [ ]:
# Define the XGB Classifier Model
model_xgb = XGBClassifier(objective='binary:logistic')
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_xgb = RFECV(estimator = model_xgb, cv=tscv, min_features_to_select=30, scoring='accuracy')
# Fitting the rfecv model to the data
model_rfecv_xgb.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_xgb.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'colsample_bytree': [0.3, 0.5, 0.7, 0.9, 1.0],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'min_child_weight': [1, 3, 5, 7]
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

random_search_xgb = RandomizedSearchCV(estimator=model_xgb, n_iter=100, param_distributions=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
random_search_xgb.fit(X_train_selected,y_train)

cv_results_xgb = random_search_xgb.cv_results_

new_row_data = ['XGB Classifier']
for scorer in scoring:
    best_validation_score = cv_results_xgb[f'mean_test_{scorer}'].max()
    i = list(cv_results_xgb[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_xgb[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

# Ridge Classifier

In [ ]:
# Define the Ridge Classifier Model
model_rc = RidgeClassifier(solver='saga', max_iter=5000, random_state=42)
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_rc = RFECV(estimator = model_rc, cv=tscv, min_features_to_select=30, scoring='accuracy')
# Fitting the rfecv model to the data
model_rfecv_rc.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_rc.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

grid_search_rc = GridSearchCV(estimator=model_rc, param_grid=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
grid_search_rc.fit(X_train_selected,y_train)

cv_results_rc = grid_search_rc.cv_results_

new_row_data = ['Ridge Classifier']
for scorer in scoring:
    best_validation_score = cv_results_rc[f'mean_test_{scorer}'].max()
    i = list(cv_results_rc[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_rc[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

# Support Vector Machine

In [ ]:
# Define the Support Vector Machine Model
model_svm = SVC(kernel='linear', random_state=42)
# Define the Recursive Feature Elimination Cross Validation
model_rfecv_svm = RFECV(estimator = model_svm, cv=tscv, min_features_to_select=30, scoring='accuracy')
# Fitting the rfecv model to the data
model_rfecv_svm.fit(X_train, y_train)
# Transforming the the training dataset to only have the selected features
X_train_selected = model_rfecv_svm.transform(X_train)
# Define a grid of hyperparameters
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': ['scale', 'auto'],  # Kernel coefficient
}
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

grid_search_svm = GridSearchCV(estimator=model_svm, param_grid=param_grid, cv=tscv, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)
grid_search_svm.fit(X_train_selected,y_train)

cv_results_svm = grid_search_svm.cv_results_

new_row_data = ['Support Vector Machine']
for scorer in scoring:
    best_validation_score = cv_results_svm[f'mean_test_{scorer}'].max()
    i = list(cv_results_svm[f'mean_test_{scorer}']).index(best_validation_score)
    train_score = list(cv_results_svm[f'mean_train_{scorer}'])[i]
    new_row_data.extend([train_score,best_validation_score])

new_row_series = pd.Series(new_row_data, index=results_df.columns)
results_df = pd.concat([results_df, pd.DataFrame(new_row_series).T], axis=0, ignore_index=True)

In [ ]:
results_df

# Choosing the Best Performing Model

# Testing the Model

# Model Performance Evaulation